In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf #numpy needs to be 1.23 or earlier in order for tf to import correctly


In [4]:
#make the spark dataframe df a pandas dataframe
income = pd.read_csv("income.csv")

In [5]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             25000 non-null  int64 
 1   workclass       23571 non-null  object
 2   education       25000 non-null  object
 3   education_num   25000 non-null  int64 
 4   marital_status  25000 non-null  object
 5   occupation      23566 non-null  object
 6   relationship    25000 non-null  object
 7   race            25000 non-null  object
 8   sex             25000 non-null  object
 9   hours_per_week  25000 non-null  int64 
 10  native_country  24563 non-null  object
 11  income          25000 non-null  object
dtypes: int64(3), object(9)
memory usage: 2.3+ MB


In [6]:
income_cat = income.dtypes[income.dtypes == 'object'].index.tolist()

In [7]:
#check the number of unique values in each column
income[income_cat].nunique()

workclass          8
education         16
marital_status     7
occupation        14
relationship       6
race               5
sex                2
native_country    40
income             2
dtype: int64

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(income[income_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(income_cat)
encode_df.head()

,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,workclass_nan,education_10th,...,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,native_country_nan,income_<=50K,income_>50K
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
income = income.merge(encode_df,left_index=True, right_index=True)
income = income.drop(income_cat,axis=1)
income.head()

,age,education_num,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,...,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,native_country_nan,income_<=50K,income_>50K
0,40,15,70,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,30,9,40,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,46,10,40,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,32,11,60,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,54,1,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
#drop income_>50K because there are 2 income columns
income.drop(['income_<=50K'], axis=1, inplace=True)


In [12]:
# Split our preprocessed data into our features and target arrays
y = income["income_>50K"].values #target is a binary 1 or 0 response
X = income.drop(["income_>50K"], axis=1).values



In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X)

# Scale the data
X_scaled = X_scaler.transform(X)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=78)

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 6


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                2625      
                                                                 
 dense_1 (Dense)             (None, 12)                312       
                                                                 
 dense_2 (Dense)             (None, 6)                 78        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 3022 (11.80 KB)
Trainable params: 3022 (11.80 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
586/586 [==============================] - 1s 731us/step - loss: 0.4392 - accuracy: 0.7795
Epoch 2/50
586/586 [==============================] - 0s 743us/step - loss: 0.3485 - accuracy: 0.8346
Epoch 3/50
586/586 [==============================] - 0s 715us/step - loss: 0.3410 - accuracy: 0.8397
Epoch 4/50
586/586 [==============================] - 0s 733us/step - loss: 0.3355 - accuracy: 0.8441
Epoch 5/50
586/586 [==============================] - 0s 801us/step - loss: 0.3330 - accuracy: 0.8468
Epoch 6/50
586/586 [==============================] - 0s 759us/step - loss: 0.3292 - accuracy: 0.8473
Epoch 7/50
586/586 [==============================] - 0s 735us/step - loss: 0.3271 - accuracy: 0.8484
Epoch 8/50
586/586 [==============================] - 0s 741us/step - loss: 0.3243 - accuracy: 0.8498
Epoch 9/50
586/586 [==============================] - 0s 747us/step - loss: 0.3226 - accuracy: 0.8516
Epoch 10/50
586/586 [==============================] - 0s 747us/step - loss: 0.320

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)


196/196 - 0s - loss: 0.4352 - accuracy: 0.8211 - 192ms/epoch - 978us/step


In [18]:

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.43517643213272095, Accuracy: 0.821120023727417
